In [7]:
import os

import gymnasium as gym
import panda_gym
import torch as th
from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor, FlattenExtractor
from huggingface_hub import notebook_login

In [8]:
env_id = "PandaPickAndPlaceDense-v3"

In [9]:
env = make_vec_env(env_id, n_envs=20)
env = VecNormalize(env,norm_obs=True, norm_reward=True)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color

In [10]:
policy_ppo = {
    "net_arch":  [dict(pi=[2048, 2048, 2048], vf=[2048, 2048,2048])],
    "activation_fn": th.nn.ReLU
}

In [11]:
model_ppo = PPO(
    policy="MultiInputPolicy",
    env=env,
    policy_kwargs=policy_ppo,
    verbose=1,
    learning_rate= 0.00015,  # Adjusted learning rate
    n_steps=2048,  # Increased n_steps for more stable updates
    batch_size=2048,  # Appropriate batch size
    n_epochs=20,  
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,  
    ent_coef=0.1,
    vf_coef=0.5,
    max_grad_norm=0.5,
    tensorboard_log="./ppo" + env_id + "/",
)

Using cuda device


/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [12]:
model_ppo.learn(15_000_000)

Logging to ./ppoPandaPickAndPlaceDense-v3/PPO_7


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 49       |
|    ep_rew_mean     | -10.3    |
|    success_rate    | 0.02     |
| time/              |          |
|    fps             | 607      |
|    iterations      | 1        |
|    time_elapsed    | 67       |
|    total_timesteps | 40960    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 47.1        |
|    ep_rew_mean          | -10.5       |
|    success_rate         | 0.06        |
| time/                   |             |
|    fps                  | 557         |
|    iterations           | 2           |
|    time_elapsed         | 146         |
|    total_timesteps      | 81920       |
| train/                  |             |
|    approx_kl            | 0.020025088 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.78     

KeyboardInterrupt: 

In [13]:
notebook_login()
!git config --global credential.helper store

In [14]:
model_ppo.save("ppo-PandaPickAndPlaceDense-v3")
env.save("PandaPickAndPlaceDense_vec_normalize_ppo.pkl")

In [15]:
eval_env = DummyVecEnv([lambda: gym.make(env_id)])
eval_env = VecNormalize.load("PandaPickAndPlaceDense_vec_normalize_ppo.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = PPO.load("ppo-PandaPickAndPlaceDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = -9.68 +/- 4.17


In [16]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"ppo-{env_id}",
    model_architecture="PPO",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Hevagog/ppo-{env_id}",
    commit_message="Initial Push",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmp3tp5x2dz/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmp3tp5x2dz/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmp3tp5x2dz/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmp3tp5x2dz/-step-0-to-step-1000.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

ℹ Pushing repo Hevagog/ppo-PandaPickAndPlaceDense-v3 to the Hugging
Face Hub


policy.optimizer.pth:   0%|          | 0.00/135M [00:00<?, ?B/s]

...

ppo-PandaPickAndPlaceDense-v3.zip:   0%|          | 0.00/203M [00:00<?, ?B/s]

policy.pth:   0%|          | 0.00/67.6M [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/3.13k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Hevagog/ppo-PandaPickAndPlaceDense-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/Hevagog/ppo-PandaPickAndPlaceDense-v3/commit/75a60ecfee119f6e0b9e3551b25d806d6451c913', commit_message='Initial Push', commit_description='', oid='75a60ecfee119f6e0b9e3551b25d806d6451c913', pr_url=None, pr_revision=None, pr_num=None)